In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
import json
from configparser import ConfigParser

config = ConfigParser()
with open("set-env") as stream:
    config.read_string("[DEFAULT]\n" + stream.read())  # This line does the trick.

pg_host=config['DEFAULT'].get("PG_HOST")
pg_uname=config['DEFAULT'].get("PG_UNAME")
pg_secret=config['DEFAULT'].get("PG_SECRET")
pg_db=config['DEFAULT'].get("PG_DB")
openai_api_endpoint=config['DEFAULT'].get("OPENAI_API_ENDPOINT")
openai_api_key=config['DEFAULT'].get("OPENAI_API_KEY")
conn_str_alchemy = f"postgresql://{pg_uname}:{pg_secret}@{pg_host}/{pg_db}"
cache_secret=config['DEFAULT'].get("CACHE_SECRET")
cache_host=config['DEFAULT'].get("CACHE_HOST")
cache_port=config['DEFAULT'].get("CACHE_PORT")
conn_str_redis = f"redis://:{cache_secret}@{cache_host}:{cache_port}/0"

In [6]:
import os
import openai
from openai import AzureOpenAI
    
client = AzureOpenAI(
    api_key=openai_api_key,  
    api_version="2023-12-01-preview",
    azure_endpoint = f"https://{openai_api_endpoint}"
)

In [16]:
openai_response = client.chat.completions.create(
    model='chat-gpt4-32k',
    messages=[
        {"role": "user", "content": "what is your name?"},
    ],
    max_tokens=256,
    temperature=0.5,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
)

openai_response.choices[0].message.content

"As an artificial intelligence, I don't have a personal name. You can simply refer to me as OpenAI."

In [7]:
import faiss
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=1280, chunk_overlap=200)
embedding = client.embeddings.create(
  input="hahahahahahahahah",
  model="embedding-ada-002"# This must match the custom deployment name you chose for your model.
  #engine="text-embedding-ada-002"
)
